In [1]:
import tensorflow as tf
x=tf.constant(3)
print(x)

tf.Tensor(3, shape=(), dtype=int32)


In [2]:
import datetime
import os
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow import keras

from tensorflow import feature_column as fc

from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Dense, DenseFeatures, concatenate)
from tensorflow.keras.callbacks import TensorBoard

print(tf.__version__)

2.6.0


In [ ]:
x2=tf.constant([[1,2,3],[4,5,6]])
tf.print(x2)

In [ ]:
y=tf.reshape(x2,[3,2])
tf.print(y)

In [ ]:
def linear_search(myList,x):
    for i in myList:
        if x == i:
            print("found the number:" + str(x))
            return
    print("not able to find:" + str(x))

arr = [1,4,6,7,9,10]
linear_search(arr,7)

In [4]:
def binary_search(myarr,x):
    mid =0
    low = 0
    high = len(myarr)-1
    while low <= high:
        mid = (high+low)//2

        if myarr[mid] < x:
            low = mid+1
        elif myarr[mid] > x:
            high = mid-1
        else:
            return mid
    return -1    
        
arr = [1,4,6,7,9,10]
print(binary_search(arr,7))

3


In [1]:
import tensorflow as tf
import numpy as np

# scalar of rank 0
myScalar = tf.constant(10000)
# vector of rank 1
myVector = tf.constant([1, 2, 3, 4, 5])
#matrix of rank 2
myMatrix = tf.constant([[1, 2, 3], [4, 5, 6]])

#print(tf.rank(myScalar))
#print(tf.rank(myVector))
print(tf.rank(myMatrix))

tf.print(myMatrix)

AttributeError: module 'tensorflow' has no attribute 'constant'

In [ ]:
import functools

import numpy as np
import tensorflow as tf

print("TensorFlow version: ",tf.version.VERSION)

In [ ]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
!head {train_file_path}

In [ ]:
# TODO 1: Add string name for label column 
LABEL_COLUMN = 'survived' 
LABELS = [0,1]

In [ ]:
def get_dataset(file_path,**kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
    return dataset

In [ ]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [ ]:
show_batch(raw_train_data)

In [ ]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [ ]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [ ]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

In [ ]:
show_batch(raw_train_data)
example_batch, labels_batch = next(iter(temp_dataset)) 

In [ ]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']
packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [ ]:
print(MEAN, STD)

In [ ]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

In [ ]:
example_batch['numeric']
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()